In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '/home/mm22d016/TransPolymer')
import torch
from sklearn.preprocessing import OneHotEncoder
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup, RobertaModel, RobertaConfig, RobertaTokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torchmetrics
from rdkit import Chem
from torchmetrics import R2Score
from PolymerSmilesTokenization import PolymerSmilesTokenizer
from copy import deepcopy

In [2]:
class Downstream_Dataset(Dataset):
    def __init__(self, dataset, tokenizer, max_token_len):
        self.tokenizer = tokenizer
        self.dataset = dataset
        self.max_token_len = max_token_len

    def __len__(self):
        self.len = len(self.dataset)
        return self.len

    def __getitem__(self, i):
        data_row = self.dataset.iloc[i]
        sd1 = data_row[0]
        sd2 = data_row[1]
        
        anion_smile = data_row[2]
        unique_AmSM = np.unique(data['Anion Smiles'].values)
        one_hot_enocder =  OneHotEncoder(categories=[unique_AmSM], sparse=False)
        one_hot_enocder.fit([[smiles] for smiles in unique_AmSM])
        anion_smile_encoding = torch.tensor(one_hot_enocder.transform([[anion_smile]])[0].astype(int))
        
        log_li = data_row[3]
        comonomer_percentage = data_row[4]/100
        approxMW = data_row[5]
        approxTg = data_row[6]
        chain_architecture = data_row[7]
        new_feats = data_row[8:self.dataset.shape[1]-1]
    
        prop = data_row[self.dataset.shape[1]-1]

        encoding1 = self.tokenizer(
            str(sd1),
            add_special_tokens=True,
            max_length=self.max_token_len,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        encoding2 = self.tokenizer(
            str(sd2),
            add_special_tokens=True,
            max_length=self.max_token_len,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        

        return dict(
            input_ids1=encoding1["input_ids"].flatten(),
            attention_mask1=encoding1["attention_mask"].flatten(),
            input_ids2=encoding2["input_ids"].flatten(),
            attention_mask2=encoding2["attention_mask"].flatten(),
            anion_smile_encoding = anion_smile_encoding,
            log_li = torch.tensor(log_li),
            comonomer_percentage = torch.tensor(comonomer_percentage),
            approxMW = torch.tensor(approxMW),
            approxTg = torch.tensor(approxTg),
            chain_architecture = torch.tensor(chain_architecture),
            new_feats = torch.tensor(new_feats),
            prop=prop
    )
    
class DataAugmentation:
    def __init__(self, aug_indicator):
        super(DataAugmentation, self).__init__()
        self.aug_indicator = aug_indicator

    """Rotate atoms to generate more SMILES"""
    def rotate_atoms(self, li, x):
        return (li[x % len(li):] + li[:x % len(li)])

    """Generate SMILES"""
    def generate_smiles(self, smiles):
        smiles_list = []
        try:
            mol = Chem.MolFromSmiles(smiles)
        except:
            mol = None
        if mol != None:
            n_atoms = mol.GetNumAtoms()
            n_atoms_list = [nat for nat in range(n_atoms)]
            if n_atoms != 0:
                for iatoms in range(n_atoms):
                    n_atoms_list_tmp = self.rotate_atoms(n_atoms_list, iatoms)  # rotate atoms' index
                    nmol = Chem.RenumberAtoms(mol, n_atoms_list_tmp)  # renumber atoms in mol
                    try:
                        smiles = Chem.MolToSmiles(nmol,
                                                  isomericSmiles=True,  # keep isomerism
                                                  kekuleSmiles=False,  # kekulize or not
                                                  rootedAtAtom=-1,  # default
                                                  canonical=False,  # canonicalize or not
                                                  allBondsExplicit=False,  #
                                                  allHsExplicit=False)  #
                    except:
                        smiles = 'None'
                    smiles_list.append(smiles)
            else:
                smiles = 'None'
                smiles_list.append(smiles)
        else:
            try:
                smiles = Chem.MolToSmiles(mol,
                                          isomericSmiles=True,  # keep isomerism
                                          kekuleSmiles=False,  # kekulize or not
                                          rootedAtAtom=-1,  # default
                                          canonical=False,  # canonicalize or not
                                          allBondsExplicit=False,  #
                                          allHsExplicit=False)  #
            except:
                smiles = 'None'
            smiles_list.append(smiles)
        smiles_array = pd.DataFrame(smiles_list).drop_duplicates().values
        # """
        if self.aug_indicator is not None:
            smiles_aug = smiles_array[1:, :]
            np.random.shuffle(smiles_aug)
            smiles_array = np.vstack((smiles_array[0, :], smiles_aug[:self.aug_indicator-1, :]))
        return smiles_array

    """SMILES Augmentation"""
    def smiles_augmentation(self, df):
        column_list = df.columns
        data_aug = np.zeros((1, df.shape[1]))
        for i in range(df.shape[0]):
            smiles = df.iloc[i, 0]
            prop = df.iloc[i, 1:]
            smiles_array = self.generate_smiles(smiles)
            if 'None' not in smiles_array:
                prop = np.tile(prop, (len(smiles_array), 1))
                data_new = np.hstack((smiles_array, prop))
                data_aug = np.vstack((data_aug, data_new))
        df_aug = pd.DataFrame(data_aug[1:, :], columns=column_list)
        return df_aug

    """Used for copolymers with two repeating units"""
    def smiles_augmentation_2(self, df):
        df_columns = df.columns
        column_list = df.columns.tolist()
        column_list_temp = deepcopy(column_list)
        column_list_temp[0] = column_list[1]
        column_list_temp[1] = column_list[0]
        df = df[column_list_temp]
        data_aug = np.zeros((1, df.shape[1]))
        for i in range(df.shape[0]):
            if df.loc[i, "Comonomer percentage"] == 100.0:
                data_new = df.values[i, :].reshape(1, -1)
                data_aug = np.vstack((data_aug, data_new))
            else:
                smiles = df.iloc[i, 0]
                prop = df.iloc[i, 1:]
                smiles_array = self.generate_smiles(smiles)
                if 'None' not in smiles_array:
                    prop = np.tile(prop, (len(smiles_array), 1))
                    data_new = np.hstack((smiles_array, prop))
                    data_aug = np.vstack((data_aug, data_new))
        data_aug_copy = deepcopy(data_aug)
        data_aug_copy[:, 0] = data_aug[:, 1]
        data_aug_copy[:, 1] = data_aug[:, 0]
        df_aug = pd.DataFrame(data_aug_copy[1:, :], columns=df_columns)
        return df_aug
    
    def combine_smiles(self, df):
        for i in range(df.shape[0]):
            if df.loc[i, "Comonomer percentage"] != 100.0:
                df.loc[i, "SMILES descriptor 1"] = df.loc[i, "SMILES descriptor 1"] + '.' + df.loc[
                    i, "SMILES descriptor 2"]
        df = df.drop(columns=['SMILES descriptor 2'])
        return df

In [3]:
class DownstreamRegression(nn.Module):
    def __init__(self, drop_rate=0.1):
        super(DownstreamRegression, self).__init__()
        self.PretrainedModel = deepcopy(PretrainedModel)
        self.PretrainedModel.resize_token_embeddings(len(tokenizer))
        
        self.smile_autoencoder = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(self.PretrainedModel.config.hidden_size, self.PretrainedModel.config.hidden_size),
            nn.SiLU(),
            nn.Linear(self.PretrainedModel.config.hidden_size, 128)
        )
        
        self.Regressor = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(211, 211),
            nn.SiLU(),
            nn.Linear(211, 1)
        )

        
        
    def forward(self, input_ids1, attention_mask1, input_ids2, attention_mask2, anion_smile_encoding, log_li, comonomer_percentage, approxMW, approxTg, chain_architecture, new_feats):
        outputs1 = self.PretrainedModel(input_ids=input_ids1, attention_mask=attention_mask1)
        logits1 = outputs1.last_hidden_state[:, 0, :]
        outputs2 = self.PretrainedModel(input_ids=input_ids2, attention_mask=attention_mask2)
        logits2 = outputs2.last_hidden_state[:, 0, :]
        logits = ((logits1*comonomer_percentage.unsqueeze(1))+((1-comonomer_percentage.unsqueeze(1))*logits2))
        print("logits",logits,logits.shape)
        smile_embeddings = self.smile_autoencoder(logits)
        print("smile_embeddings",smile_embeddings,smile_embeddings.shape)
        input_to_regressor = torch.cat([smile_embeddings, anion_smile_encoding, log_li.unsqueeze(1), approxMW.unsqueeze(1), approxTg.unsqueeze(1), chain_architecture.unsqueeze(1), new_feats], dim=1).clone().detach()
        print("input_to_regressor",input_to_regressor,input_to_regressor.shape)
        output = self.Regressor(input_to_regressor)
        
        return output


def train(model, loss_fn, train_dataloader, device):

    #model.train()

    for step, batch in enumerate(train_dataloader):
        input_ids1 = batch["input_ids1"].to(device)

        attention_mask1 = batch["attention_mask1"].to(device)
        #print("attention_mask1",attention_mask1,attention_mask1.shape)
        input_ids2 = batch["input_ids2"].to(device)
        #print("input_ids2",input_ids2,input_ids2.shape)
        attention_mask2 = batch["attention_mask2"].to(device)
        #print("attention_mask2",attention_mask2,attention_mask2.shape)
        anion_smile_encoding = batch["anion_smile_encoding"].to(device)
        #print("anion_smile_encoding",anion_smile_encoding,anion_smile_encoding.shape)
        log_li = batch["log_li"].to(device)
        #print("log_li",log_li,log_li.shape)
        comonomer_percentage = batch["comonomer_percentage"].to(device)
        #print("comonomer_percentage",comonomer_percentage,comonomer_percentage.shape)
        approxMW = batch["approxMW"].to(device)
        #print("approxMW",approxMW,approxMW.shape)
        approxTg = batch["approxTg"].to(device)
        #print("approxTg",approxTg,approxTg.shape)
        chain_architecture = batch["chain_architecture"].to(device)
        #print("chain_architecture",chain_architecture,chain_architecture.shape)
        new_feats = batch["new_feats"].to(device)
        #print("new_feats",new_feats,new_feats.shape)
        prop = batch["prop"].to(device).float()
        #print("prop",prop,prop.shape)
        outputs = model(input_ids1, attention_mask1, input_ids2, attention_mask2, anion_smile_encoding, log_li, comonomer_percentage, approxMW, approxTg, chain_architecture, new_feats).float()
        print("End of Batch ------------------------------------------------------------------------------------")

        #loss = loss_fn(outputs.squeeze(), prop.squeeze())
        #loss.backward()
        #optimizer.step()
        #scheduler.step()

    return None


In [4]:
def test(model, loss_fn, train_dataloader, test_dataloader, scaler_target, device):

    r2score = R2Score()
    train_loss = 0
    test_loss = 0
    # count = 0
    model.eval()
    with torch.no_grad():
        train_pred, train_true, test_pred, test_true = torch.tensor([]), torch.tensor([]), torch.tensor(
            []), torch.tensor([])

        print("Train_dataloader")
        for step, batch in enumerate(train_dataloader):
            input_ids1 = batch["input_ids1"].to(device)
            attention_mask1 = batch["attention_mask1"].to(device)
            input_ids2 = batch["input_ids2"].to(device)
            attention_mask2 = batch["attention_mask2"].to(device)
            anion_smile_encoding = batch["anion_smile_encoding"].to(device)
            log_li = batch["log_li"].to(device)
            comonomer_percentage = batch["comonomer_percentage"].to(device)
            approxMW = batch["approxMW"].to(device)
            approxTg = batch["approxTg"].to(device)
            chain_architecture = batch["chain_architecture"].to(device)
            new_feats = batch["new_feats"].to_device()
            prop = batch["prop"].to(device).float()
            outputs = model(input_ids1, attention_mask1, input_ids2, attention_mask2, anion_smile_encoding, log_li, comonomer_percentage, approxMW, approxTg, chain_architecture, new_feats).float()
            outputs = torch.from_numpy(scaler_target.inverse_transform(outputs.cpu().reshape(-1, 1)))
            prop = torch.from_numpy(scaler_target.inverse_transform(prop.cpu().reshape(-1, 1)))
            loss = loss_fn(outputs.squeeze(), prop.squeeze())
            train_loss += loss.item() * len(prop)
            train_pred = torch.cat([train_pred.to(device), outputs.to(device)])
            train_true = torch.cat([train_true.to(device), prop.to(device)])
            print("End of Batch ------------------------------------------")

        train_loss = train_loss / len(train_pred.flatten())
        r2_train = r2score(train_pred.flatten().to("cpu"), train_true.flatten().to("cpu")).item()

        print("Test_dataloader")
        for step, batch in enumerate(test_dataloader):
            input_ids1 = batch["input_ids1"].to(device)
            attention_mask1 = batch["attention_mask1"].to(device)
            input_ids2 = batch["input_ids2"].to(device)
            attention_mask2 = batch["attention_mask2"].to(device)
            anion_smile_encoding = batch["anion_smile_encoding"].to(device)
            log_li = batch["log_li"].to(device)
            comonomer_percentage = batch["comonomer_percentage"].to(device)
            approxMW = batch["approxMW"].to(device)
            approxTg = batch["approxTg"].to(device)
            chain_architecture = batch["chain_architecture"].to(device)
            new_feats = batch["new_feats"].to_device()
            prop = batch["prop"].to(device).float()
            outputs = model(input_ids1, attention_mask1, input_ids2, attention_mask2, anion_smile_encoding, log_li, comonomer_percentage, approxMW, approxTg, chain_architecture, new_feats).float()
            outputs = torch.from_numpy(scaler_target.inverse_transform(outputs.cpu().reshape(-1, 1)))
            prop = torch.from_numpy(scaler_target.inverse_transform(prop.cpu().reshape(-1, 1)))
            loss = loss_fn(outputs.squeeze(), prop.squeeze())
            test_loss += loss.item() * len(prop)
            test_pred = torch.cat([test_pred.to(device), outputs.to(device)])
            test_true = torch.cat([test_true.to(device), prop.to(device)])
            print("End of Batch ------------------------------------------")
            
        test_loss = test_loss / len(test_pred.flatten())
        r2_test = r2score(test_pred.flatten().to("cpu"), test_true.flatten().to("cpu")).item()
    
    return None

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vocab_sup = pd.read_csv('../data/vocab/vocab_sup_PE_II.csv', header=None).values.flatten().tolist()
PretrainedModel = RobertaModel.from_pretrained('../ckpt/pretrain.pt')
tokenizer = PolymerSmilesTokenizer.from_pretrained("roberta-base", max_len=411)
tokenizer.add_tokens(vocab_sup)
PretrainedModel.config.hidden_dropout_prob = 0.1
PretrainedModel.config.attention_probs_dropout_prob = 0.1
max_token_len = 411
model = DownstreamRegression(drop_rate=0.1).to(device)
model = model.double()
loss_fn = nn.MSELoss()
torch.cuda.empty_cache()

Some weights of the model checkpoint at ../ckpt/pretrain.pt were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ../ckpt/pretrain.pt and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer clas

In [6]:
data = pd.read_csv('../data/new_feat_Li.csv')
data['chain architecture'] = data['chain architecture'].replace({'S_1': 0, 'S_2': 1})
train_data, test_data = train_test_split(data, test_size=0.2, random_state=1)
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)
train_data

,SMILES descriptor 1,SMILES descriptor 2,Anion Smiles,log Li:functional group,Comonomer percentage,approxMW(kDa),approxTg,chain architecture,Comonomer 1 CIC1,Comonomer 1 JGI8,...,Comonomer 2 RDF100m,Comonomer 2 SMR_VSA7,Comonomer 2 SlogP_VSA10,Comonomer 2 SlogP_VSA12,Comonomer 2 SlogP_VSA6,Comonomer 2 TDB10s,Comonomer 2 VSA_EState10,Comonomer 2 fr_aryl_methyl,drying vacuum,logCond60
0,*CC(C*)C#N,*CC(C*)C#N,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-0.778151,100.0,170.000000,80.000000,0,1.804290,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,2,-6.690204
1,CC1=C(C)C(COCP(=S)(CO*)CC2=CC=CC=C2)=C(C)C(C)=...,CC1=C(C)C(COCP(=S)(CO*)CC2=CC=CC=C2)=C(C)C(C)=...,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-0.255273,100.0,9.600000,171.000000,0,3.010579,0.012467,...,3.508434e+00,69.778911,0.000000,6.037803,36.398202,18.374741,5.976773,2,2,-4.838063
2,*CNC*,*CNC*,C(F)(F)(F)S(=O)(=O)[O-],-1.176091,100.0,122664.000000,-17.328918,0,1.750978,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,2,-5.541343
3,COCCOCC(C*)O*,*OC(*)=O,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.477121,50.0,17.000000,-55.000000,1,2.959037,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,2,-4.528742
4,*CCO*,*CCO*,F[P-](F)(F)(F)(F)F,-1.204120,100.0,4000.000000,-44.000000,0,1.945531,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,-6.656500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,*CCO*,*CCO*,C(F)(F)(F)S(=O)(=O)[O-],-1.079181,100.0,13.207547,-42.000000,0,1.945531,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,2,-4.394700
212,*CCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOC*,*C1OC(=O)OC1*,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.204120,50.0,8.100000,-35.600000,1,4.858178,0.000950,...,0.000000e+00,0.000000,4.794537,0.000000,0.000000,0.000000,0.000000,0,0,-3.629528
213,*CCCCCC(=O)O*,*CCCCCC(=O)O*,[O-]Cl(=O)(=O)=O,-1.469449,100.0,65.000000,-55.000000,0,2.250977,0.000000,...,6.220000e-93,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,2,-4.607459
214,CCOCOCCN(*)CC(*)=O,CCOCOCCN(*)CC(*)=O,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.070581,100.0,3.242800,9.900000,1,2.309918,0.008230,...,1.710878e-01,0.000000,0.000000,0.000000,0.000000,22.410993,0.000000,0,2,-4.822385


In [7]:
test_data

,SMILES descriptor 1,SMILES descriptor 2,Anion Smiles,log Li:functional group,Comonomer percentage,approxMW(kDa),approxTg,chain architecture,Comonomer 1 CIC1,Comonomer 1 JGI8,...,Comonomer 2 RDF100m,Comonomer 2 SMR_VSA7,Comonomer 2 SlogP_VSA10,Comonomer 2 SlogP_VSA12,Comonomer 2 SlogP_VSA6,Comonomer 2 TDB10s,Comonomer 2 VSA_EState10,Comonomer 2 fr_aryl_methyl,drying vacuum,logCond60
0,*CCO*,COCCOCCOCC(C*)O*,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.698970,91.00,277.777778,-55.45000,1,1.945531,0.000000,...,7.948876e-02,0.000000,0.000000,0.000000,0.000000,19.149043,0.000000,0,3,-3.460793
1,CC(CO*)OC(=O)CCCC(*)=O,CC(CO*)OC(=O)CCCC(*)=O,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.096910,100.00,8.800000,-16.00000,0,2.357806,0.008230,...,3.233966e-01,0.000000,0.000000,0.000000,0.000000,15.851587,0.000000,0,2,-4.661223
2,*CCO*,*CC(COCC=C)O*,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.422905,76.70,7.900000,-69.00000,0,1.945531,0.000000,...,1.250000e-05,12.654956,0.000000,0.000000,12.654956,0.000000,0.000000,0,3,-3.652059
3,*CC(C*)C#N,*CC(C*)C#N,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.079181,100.00,170.000000,80.00000,0,1.804290,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,2,-6.735954
4,*CCOC(=O)O*,*CCO*,[B-](F)(F)(F)F,-0.135672,95.20,37.000000,-69.00000,0,1.625815,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,2,-4.340000
5,*CCOC(=O)O*,*CCO*,C(F)(F)(F)S(=O)(=O)[O-],-0.083089,95.20,37.000000,-7.00000,0,1.625815,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,2,-6.700000
6,*OCC(COCC=C)OC(=O)COCC(*)=O,*OCC(COCC=C)OC(=O)COCC(*)=O,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.154902,100.00,8.900000,4.00000,1,2.112984,0.006173,...,3.650635e-02,12.654956,0.000000,0.000000,12.654956,23.910774,0.000000,0,2,-4.948854
7,C[Si](*)(CCSCCCC(=O)NCCCN1C=CN=C1)O*,C[Si](*)(CCSCCCC(=O)NCCCN1C=CN=C1)O*,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.823909,100.00,110943.600000,-6.00000,1,1.908336,0.009575,...,0.000000e+00,18.721007,0.000000,11.761885,18.721007,0.000000,1.818517,0,3,-6.446174
8,*CCO*,CC(C*)O*,[O-]Cl(=O)(=O)=O,-0.958607,47.00,5.000000,-7.00000,0,1.945531,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,2,-5.721043
9,C[Si](*)(CCSCCCC(=O)NCCCN1C=CN=C1)O*,C[Si](*)(CCSCCCCCCCC1=CC=CC=C1)O*,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.158015,72.00,67760.000000,-13.00000,1,1.908336,0.009575,...,0.000000e+00,35.895287,0.000000,11.761885,30.331835,0.000000,2.007703,1,3,-5.971410


In [8]:
DataAug = DataAugmentation(None)
train_data = DataAug.smiles_augmentation(train_data)
train_data

In [9]:
train_data = DataAug.smiles_augmentation_2(train_data)
train_data

In [10]:
scaler_feat = StandardScaler()
train_data[['approxMW(kDa)','approxTg','Comonomer 1 CIC1', 'Comonomer 1 JGI8', 'Comonomer 1 MQNs_16','Comonomer 1 NumAromaticRings', 'Comonomer 1 RDF120u', 'Comonomer 1 RDF130m', 'Comonomer 1 RDF65u', 'Comonomer 1 RDF95m', 'Comonomer 1 RDF95u', 'Comonomer 1 SMR_VSA10', 'Comonomer 1 SMR_VSA7', 'Comonomer 1 SlogP_VSA12', 'Comonomer 1 SlogP_VSA6', 'Comonomer 1 fr_aryl_methyl', 'Comonomer 1 qed', 'Comonomer 2 AVP-0', 'Comonomer 2 GATS3m', 'Comonomer 2 GATS5v', 'Comonomer 2 MATS3s', 'Comonomer 2 MATS4c', 'Comonomer 2 MQNs_16', 'Comonomer 2 MQNs_5', 'Comonomer 2 NumAromaticCarbocycles', 'Comonomer 2 NumAromaticRings', 'Comonomer 2 RDF100m', 'Comonomer 2 SMR_VSA7', 'Comonomer 2 SlogP_VSA10', 'Comonomer 2 SlogP_VSA12', 'Comonomer 2 SlogP_VSA6', 'Comonomer 2 TDB10s', 'Comonomer 2 VSA_EState10', 'Comonomer 2 fr_aryl_methyl','drying vacuum']] = scaler_feat.fit_transform(train_data[['approxMW(kDa)','approxTg','Comonomer 1 CIC1', 'Comonomer 1 JGI8', 'Comonomer 1 MQNs_16','Comonomer 1 NumAromaticRings', 'Comonomer 1 RDF120u', 'Comonomer 1 RDF130m', 'Comonomer 1 RDF65u', 'Comonomer 1 RDF95m', 'Comonomer 1 RDF95u', 'Comonomer 1 SMR_VSA10', 'Comonomer 1 SMR_VSA7', 'Comonomer 1 SlogP_VSA12', 'Comonomer 1 SlogP_VSA6', 'Comonomer 1 fr_aryl_methyl', 'Comonomer 1 qed', 'Comonomer 2 AVP-0', 'Comonomer 2 GATS3m', 'Comonomer 2 GATS5v', 'Comonomer 2 MATS3s', 'Comonomer 2 MATS4c', 'Comonomer 2 MQNs_16', 'Comonomer 2 MQNs_5', 'Comonomer 2 NumAromaticCarbocycles', 'Comonomer 2 NumAromaticRings', 'Comonomer 2 RDF100m', 'Comonomer 2 SMR_VSA7', 'Comonomer 2 SlogP_VSA10', 'Comonomer 2 SlogP_VSA12', 'Comonomer 2 SlogP_VSA6', 'Comonomer 2 TDB10s', 'Comonomer 2 VSA_EState10', 'Comonomer 2 fr_aryl_methyl','drying vacuum']])
test_data[['approxMW(kDa)','approxTg','Comonomer 1 CIC1', 'Comonomer 1 JGI8', 'Comonomer 1 MQNs_16','Comonomer 1 NumAromaticRings', 'Comonomer 1 RDF120u', 'Comonomer 1 RDF130m', 'Comonomer 1 RDF65u', 'Comonomer 1 RDF95m', 'Comonomer 1 RDF95u', 'Comonomer 1 SMR_VSA10', 'Comonomer 1 SMR_VSA7', 'Comonomer 1 SlogP_VSA12', 'Comonomer 1 SlogP_VSA6', 'Comonomer 1 fr_aryl_methyl', 'Comonomer 1 qed', 'Comonomer 2 AVP-0', 'Comonomer 2 GATS3m', 'Comonomer 2 GATS5v', 'Comonomer 2 MATS3s', 'Comonomer 2 MATS4c', 'Comonomer 2 MQNs_16', 'Comonomer 2 MQNs_5', 'Comonomer 2 NumAromaticCarbocycles', 'Comonomer 2 NumAromaticRings', 'Comonomer 2 RDF100m', 'Comonomer 2 SMR_VSA7', 'Comonomer 2 SlogP_VSA10', 'Comonomer 2 SlogP_VSA12', 'Comonomer 2 SlogP_VSA6', 'Comonomer 2 TDB10s', 'Comonomer 2 VSA_EState10', 'Comonomer 2 fr_aryl_methyl','drying vacuum']] = scaler_feat.transform(test_data[['approxMW(kDa)','approxTg','Comonomer 1 CIC1', 'Comonomer 1 JGI8', 'Comonomer 1 MQNs_16','Comonomer 1 NumAromaticRings', 'Comonomer 1 RDF120u', 'Comonomer 1 RDF130m', 'Comonomer 1 RDF65u', 'Comonomer 1 RDF95m', 'Comonomer 1 RDF95u', 'Comonomer 1 SMR_VSA10', 'Comonomer 1 SMR_VSA7', 'Comonomer 1 SlogP_VSA12', 'Comonomer 1 SlogP_VSA6', 'Comonomer 1 fr_aryl_methyl', 'Comonomer 1 qed', 'Comonomer 2 AVP-0', 'Comonomer 2 GATS3m', 'Comonomer 2 GATS5v', 'Comonomer 2 MATS3s', 'Comonomer 2 MATS4c', 'Comonomer 2 MQNs_16', 'Comonomer 2 MQNs_5', 'Comonomer 2 NumAromaticCarbocycles', 'Comonomer 2 NumAromaticRings', 'Comonomer 2 RDF100m', 'Comonomer 2 SMR_VSA7', 'Comonomer 2 SlogP_VSA10', 'Comonomer 2 SlogP_VSA12', 'Comonomer 2 SlogP_VSA6', 'Comonomer 2 TDB10s', 'Comonomer 2 VSA_EState10', 'Comonomer 2 fr_aryl_methyl','drying vacuum']])

/tmp/ipykernel_70487/3966178511.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[['approxMW(kDa)','approxTg','Comonomer 1 CIC1', 'Comonomer 1 JGI8', 'Comonomer 1 MQNs_16','Comonomer 1 NumAromaticRings', 'Comonomer 1 RDF120u', 'Comonomer 1 RDF130m', 'Comonomer 1 RDF65u', 'Comonomer 1 RDF95m', 'Comonomer 1 RDF95u', 'Comonomer 1 SMR_VSA10', 'Comonomer 1 SMR_VSA7', 'Comonomer 1 SlogP_VSA12', 'Comonomer 1 SlogP_VSA6', 'Comonomer 1 fr_aryl_methyl', 'Comonomer 1 qed', 'Comonomer 2 AVP-0', 'Comonomer 2 GATS3m', 'Comonomer 2 GATS5v', 'Comonomer 2 MATS3s', 'Comonomer 2 MATS4c', 'Comonomer 2 MQNs_16', 'Comonomer 2 MQNs_5', 'Comonomer 2 NumAromaticCarbocycles', 'Comonomer 2 NumAromaticRings', 'Comonomer 2 RDF100m', 'Comonomer 2 SMR_VSA7', 'Como

In [11]:
scaler_target = StandardScaler()
train_data.iloc[:, train_data.shape[1]-1] = scaler_target.fit_transform(train_data.iloc[:, train_data.shape[1]-1].values.reshape(-1, 1))
test_data.iloc[:, test_data.shape[1]-1] = scaler_target.transform(test_data.iloc[:, test_data.shape[1]-1].values.reshape(-1, 1))

In [13]:
test_data

,SMILES descriptor 1,SMILES descriptor 2,Anion Smiles,log Li:functional group,Comonomer percentage,approxMW(kDa),approxTg,chain architecture,Comonomer 1 CIC1,Comonomer 1 JGI8,...,Comonomer 2 RDF100m,Comonomer 2 SMR_VSA7,Comonomer 2 SlogP_VSA10,Comonomer 2 SlogP_VSA12,Comonomer 2 SlogP_VSA6,Comonomer 2 TDB10s,Comonomer 2 VSA_EState10,Comonomer 2 fr_aryl_methyl,drying vacuum,logCond60
0,*CCO*,COCCOCCOCC(C*)O*,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.698970,91.00,-0.387984,-0.805523,1,-0.556557,-0.726758,...,-0.271044,-0.330340,-0.525625,-0.372231,-0.388859,0.823197,-0.324604,-0.223933,1.072718,1.266018
1,CC(CO*)OC(=O)CCCC(*)=O,CC(CO*)OC(=O)CCCC(*)=O,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.096910,100.00,-0.395315,-0.025342,0,-0.085529,1.334310,...,0.032122,-0.330340,-0.525625,-0.372231,-0.388859,0.589743,-0.324604,-0.223933,0.014695,0.279220
2,*CCO*,*CC(COCC=C)O*,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.422905,76.70,-0.395339,-1.073494,0,-0.556557,-0.726758,...,-0.369830,0.556070,-0.525625,-0.372231,1.057407,-0.532523,-0.324604,-0.223933,1.072718,1.108790
3,*CC(C*)C#N,*CC(C*)C#N,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.079181,100.00,-0.390921,1.873198,0,-0.717925,-0.726758,...,-0.369845,-0.330340,-0.525625,-0.372231,-0.388859,-0.532523,-0.324604,-0.223933,0.014695,-1.426285
4,*CCOC(=O)O*,*CCO*,[B-](F)(F)(F)F,-0.135672,95.20,-0.394546,-1.073494,0,-0.921835,-0.726758,...,-0.369845,-0.330340,-0.525625,-0.372231,-0.388859,-0.532523,-0.324604,-0.223933,0.014695,0.543277
5,*CCOC(=O)O*,*CCO*,C(F)(F)(F)S(=O)(=O)[O-],-0.083089,95.20,-0.394546,0.152646,0,-0.921835,-0.726758,...,-0.369845,-0.330340,-0.525625,-0.372231,-0.388859,-0.532523,-0.324604,-0.223933,0.014695,-1.396730
6,*OCC(COCC=C)OC(=O)COCC(*)=O,*OCC(COCC=C)OC(=O)COCC(*)=O,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.154902,100.00,-0.395312,0.370187,1,-0.365241,0.819043,...,-0.324470,0.556070,-0.525625,-0.372231,1.057407,1.160319,-0.324604,-0.223933,0.014695,0.042777
7,C[Si](*)(CCSCCCC(=O)NCCCN1C=CN=C1)O*,C[Si](*)(CCSCCCC(=O)NCCCN1C=CN=C1)O*,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.823909,100.00,2.627992,0.172423,1,-0.599053,1.670990,...,-0.369845,0.980964,-0.525625,2.963018,1.750663,-0.532523,1.159810,-0.223933,1.072718,-1.188075
8,*CCO*,CC(C*)O*,[O-]Cl(=O)(=O)=O,-0.958607,47.00,-0.395418,0.152646,0,-0.556557,-0.726758,...,-0.369845,-0.330340,-0.525625,-0.372231,-0.388859,-0.532523,-0.324604,-0.223933,0.014695,-0.591991
9,C[Si](*)(CCSCCCC(=O)NCCCN1C=CN=C1)O*,C[Si](*)(CCSCCCCCCCC1=CC=CC=C1)O*,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.158015,72.00,1.451109,0.033987,1,-0.599053,1.670990,...,-0.369845,2.183927,-0.525625,2.963018,3.077601,-0.532523,1.314237,2.321829,1.072718,-0.797802


In [14]:
train_dataset = Downstream_Dataset(train_data, tokenizer, 411)
test_dataset = Downstream_Dataset(test_data, tokenizer, 411)

In [15]:
train_dataloader = DataLoader(train_dataset, 2, shuffle=False, num_workers=8)
test_dataloader = DataLoader(test_dataset, 2, shuffle=False, num_workers=8)

In [16]:
train(model, loss_fn, train_dataloader, device)

logits tensor([[-0.5995, -0.5397,  0.0674,  ..., -0.2653, -0.1414,  1.5422],
        [ 0.6032, -2.1316, -0.1725,  ..., -1.5678, -0.5253, -0.3527]],
       device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>) torch.Size([2, 768])
smile_embeddings tensor([[-1.7507e-02,  3.0780e-01, -3.6170e-01, -1.5713e-01,  2.2114e-01,
          3.3442e-03, -1.2810e-01,  2.0819e-02, -2.0039e-02,  1.0930e-01,
         -2.2876e-01, -5.5523e-02, -1.2781e-01,  2.1920e-01, -1.8712e-01,
         -4.1816e-02, -4.1408e-01,  1.9307e-01,  1.7334e-01,  2.9918e-01,
          6.9719e-02,  3.4996e-01,  1.2484e-01, -3.5378e-01,  1.5688e-02,
         -1.4233e-01, -8.6232e-02,  1.9506e-01,  2.1952e-01,  2.3682e-03,
         -1.3299e-01, -2.2821e-01, -1.1878e-01, -1.0824e-01,  2.4553e-01,
          1.1942e-01, -1.8458e-02, -9.9897e-02,  1.3426e-01, -1.3144e-01,
          1.4590e-01,  3.0798e-01,  5.9213e-01, -2.2333e-02, -5.8445e-02,
         -9.7453e-02,  2.2535e-01,  3.7430e-01, -1.3734e-01,  3.1384e-01,
     

smile_embeddings tensor([[-0.0403,  0.2243, -0.2317, -0.0624,  0.2025, -0.0994, -0.0743,  0.1363,
          0.0679,  0.1109, -0.3074,  0.0077,  0.0303,  0.0335, -0.2377, -0.1017,
         -0.3056,  0.1853,  0.0501,  0.4205,  0.1553,  0.2113,  0.1102, -0.4130,
          0.1476, -0.1027, -0.0901,  0.2963,  0.0419,  0.0747,  0.0167, -0.1760,
         -0.1342, -0.0540,  0.3841,  0.0825, -0.1373, -0.0359,  0.1501, -0.1457,
          0.1682,  0.1647,  0.5669,  0.1133,  0.0549, -0.1079,  0.3650,  0.3363,
         -0.1908,  0.3146,  0.4307,  0.1116, -0.0436, -0.1133,  0.2214,  0.1240,
         -0.1689, -0.0899, -0.2309, -0.0463,  0.0827,  0.1406,  0.1869,  0.0743,
          0.1339, -0.0710,  0.3597, -0.1587, -0.0118, -0.1146, -0.0638, -0.0262,
         -0.4249,  0.3877, -0.1336,  0.2301,  0.1767,  0.2679, -0.3269,  0.0444,
         -0.2698,  0.0396,  0.0526,  0.3878,  0.0350, -0.2360, -0.1761,  0.0853,
          0.1199,  0.1017,  0.3797,  0.3076, -0.1910,  0.2944,  0.2518, -0.1197,
          0

logits tensor([[ 0.6099, -1.1832, -0.4592,  ..., -1.0539, -0.0378,  0.8072],
        [ 0.1294, -1.8128, -0.1654,  ..., -1.4135,  0.1382,  0.7863]],
       device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>) torch.Size([2, 768])
smile_embeddings tensor([[-0.1566,  0.0357, -0.2511, -0.2987,  0.1182,  0.1437, -0.2654, -0.1449,
         -0.1384,  0.0995,  0.0253,  0.1021, -0.2399,  0.0627,  0.0063, -0.1191,
         -0.2659,  0.0425, -0.0231,  0.2825, -0.1453,  0.0248,  0.1832, -0.2609,
          0.1144, -0.0761, -0.0394,  0.0715,  0.0638, -0.1975,  0.1701, -0.1350,
         -0.1756,  0.2026,  0.0697,  0.0861,  0.2725, -0.0073, -0.0028, -0.0750,
          0.1087, -0.0200,  0.0416, -0.0632,  0.0061, -0.1194, -0.0631,  0.1175,
         -0.0535, -0.0375,  0.1237, -0.1197,  0.0541, -0.2592,  0.2299,  0.1310,
         -0.0786, -0.3012, -0.0227,  0.1082,  0.2599, -0.0070, -0.0607, -0.1256,
          0.2430,  0.0901,  0.1593, -0.0701, -0.2133, -0.2954, -0.2642, -0.2045,
          0.0169

logits tensor([[ 0.8386, -0.5247,  0.1221,  ..., -0.0886, -0.1060,  2.2836],
        [-0.4638, -0.8124,  0.1468,  ..., -0.4924,  0.4880,  1.8449]],
       device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>) torch.Size([2, 768])
smile_embeddings tensor([[-0.0492, -0.0556, -0.0903, -0.0189,  0.0486,  0.0387, -0.1742, -0.0595,
          0.0290,  0.1070, -0.0327, -0.0252, -0.1094,  0.1928, -0.3529, -0.2534,
         -0.3234,  0.1900, -0.2032,  0.3310, -0.0056,  0.2842,  0.3372, -0.4240,
         -0.0794, -0.0254, -0.0709,  0.0437,  0.0401,  0.0378,  0.1311, -0.2328,
          0.2076,  0.2239,  0.2800,  0.0378, -0.1267, -0.1358,  0.2826, -0.0403,
          0.3549,  0.1863,  0.3543, -0.0153, -0.2933, -0.3421,  0.1342,  0.5220,
          0.3143,  0.0397,  0.1550,  0.0464, -0.0455, -0.1974,  0.0428,  0.1946,
         -0.3277, -0.0926, -0.2478,  0.2845, -0.0130,  0.2969,  0.2560, -0.0593,
          0.2506,  0.1290,  0.2943, -0.1713, -0.0746, -0.3298, -0.5100, -0.0694,
         -0.3147

logits tensor([[-0.2201, -1.2003, -0.1129,  ..., -0.4003,  0.5902,  1.1035],
        [ 0.1387, -1.5587, -0.0978,  ..., -0.2143, -0.2845,  0.7609]],
       device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>) torch.Size([2, 768])
smile_embeddings tensor([[-2.4673e-02,  3.7002e-01, -4.1227e-01,  1.8881e-02,  2.4161e-01,
         -9.0571e-02,  4.9651e-02, -4.6838e-02, -8.1031e-02,  1.0413e-01,
         -2.1596e-01,  5.8511e-02, -3.1360e-02,  1.1099e-01, -2.3279e-01,
          1.9666e-02, -2.6774e-01,  2.5359e-01,  1.9786e-01,  5.1087e-01,
          1.6142e-01,  3.9306e-01,  1.2489e-01, -3.7283e-01,  1.2591e-01,
          2.4763e-02, -3.7741e-02,  1.8716e-01,  1.1234e-01,  1.0159e-01,
         -7.5299e-02, -1.4351e-01, -4.5999e-02, -1.1993e-01,  2.7004e-01,
          9.9644e-02, -1.6272e-01, -1.5030e-01,  5.5808e-02, -3.5710e-02,
          8.8550e-02,  2.7366e-01,  5.1305e-01,  1.4478e-01, -8.0500e-02,
         -1.7253e-01,  1.3414e-01,  2.9143e-01, -2.0097e-01,  2.6030e-01,
     

...

logits tensor([[-0.1431, -1.2431, -0.0626,  ..., -0.4679,  0.7815,  1.0820],
        [-0.2188, -0.9537, -0.9130,  ...,  0.4257,  0.5275,  1.5223]],
       device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>) torch.Size([2, 768])
smile_embeddings tensor([[-3.7268e-02,  3.5143e-01, -4.0595e-01, -9.4787e-03,  2.0112e-01,
         -1.5444e-01, -8.2956e-03,  4.4189e-02, -4.1152e-02,  2.9178e-01,
         -2.4046e-01,  7.0410e-02,  2.5862e-02,  9.2725e-02, -2.9567e-01,
          4.0094e-02, -2.0333e-01,  1.2849e-01,  1.7886e-01,  4.3197e-01,
          2.0758e-01,  5.8328e-01,  1.4372e-01, -3.4904e-01,  1.1648e-01,
         -1.0673e-01, -5.8010e-02,  2.7180e-01,  1.6949e-01,  1.6156e-01,
         -1.7179e-01, -6.9630e-02, -1.1749e-01, -6.2877e-02,  1.5566e-01,
          1.5643e-01, -6.9837e-02,  1.1018e-02,  1.6497e-01,  7.0118e-02,
          1.0436e-01,  2.3805e-01,  4.5788e-01,  3.9134e-02, -1.5392e-01,
         -3.0930e-02,  1.8356e-01,  2.5505e-01, -1.7167e-01,  3.4329e-01,
     

input_to_regressor tensor([[-1.1349e-01,  9.5216e-02, -2.5067e-01, -4.3327e-01, -1.2894e-02,
          1.7923e-01, -1.8634e-01, -8.6593e-02, -4.5398e-02,  5.5300e-02,
         -2.0621e-02,  1.3855e-01, -2.8289e-01,  6.3326e-02, -1.4678e-01,
          4.6312e-02, -1.5076e-01,  4.0434e-02, -2.1413e-02,  3.6053e-01,
         -2.5494e-01, -1.0200e-01,  2.5409e-01, -2.4784e-01,  1.5660e-01,
          6.6845e-02, -8.2213e-02,  4.6166e-02, -7.8558e-02, -6.7083e-02,
          1.1183e-01, -6.6138e-02, -2.0872e-01,  5.4761e-02, -7.6379e-02,
          6.5738e-02,  4.2392e-01, -1.3725e-01,  4.3868e-02, -1.8316e-01,
          1.0196e-01,  3.3062e-02,  2.7255e-02, -1.9839e-01,  1.2109e-01,
         -1.0654e-01, -7.1707e-02,  1.8913e-01,  2.8576e-02, -4.3705e-02,
          1.7067e-01, -1.0800e-01, -1.4586e-02, -2.5904e-01,  2.7868e-01,
          1.2420e-01, -2.3754e-01, -3.6685e-01,  1.3897e-01,  1.4203e-01,
          2.9937e-01, -9.8130e-03, -1.4064e-01, -3.5396e-01,  2.8241e-01,
          9.7810e-0

smile_embeddings tensor([[-4.7985e-02,  2.8997e-01, -3.8804e-01,  2.0606e-01,  1.3515e-01,
         -1.1381e-01, -1.2048e-01, -6.0610e-03, -4.9587e-02,  1.3980e-01,
         -2.1809e-01,  1.0955e-01, -1.5646e-02,  1.1312e-01, -2.4220e-01,
          3.5192e-02, -2.5896e-01,  2.0560e-01,  8.0952e-02,  3.7234e-01,
          1.9976e-01,  3.7301e-01,  2.5382e-01, -3.6032e-01,  1.1906e-01,
         -6.4650e-02, -1.0603e-01,  2.7404e-01,  5.6358e-02,  1.1093e-01,
         -8.9265e-02, -1.2354e-01,  3.7774e-02, -8.9006e-02,  3.7899e-01,
          1.3365e-01, -1.1959e-01, -4.0826e-03,  5.2195e-02, -2.4633e-02,
          2.2358e-01,  3.1568e-01,  4.8479e-01,  1.2115e-01, -1.1471e-01,
         -7.5406e-02,  2.5085e-01,  2.9834e-01, -2.0981e-01,  2.8312e-01,
          2.2861e-01,  1.0219e-01,  5.0753e-02,  8.8429e-02,  3.0091e-01,
          9.7293e-02,  3.5154e-02, -4.8129e-02, -2.7165e-02, -6.8123e-02,
         -3.9976e-02,  1.7783e-01,  9.8053e-02, -5.2546e-02,  1.2867e-01,
         -1.2322e-01,

logits tensor([[ 0.8386, -0.5247,  0.1221,  ..., -0.0886, -0.1060,  2.2836],
        [-0.2966, -0.8822, -0.8697,  ...,  0.3624,  0.5809,  1.6490]],
       device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>) torch.Size([2, 768])
smile_embeddings tensor([[-0.1425, -0.0168, -0.1144, -0.1211, -0.0216,  0.0854, -0.0631, -0.0647,
          0.1680, -0.0527, -0.0550,  0.0023, -0.2448,  0.1323, -0.2078, -0.2513,
         -0.2600,  0.1919, -0.0906,  0.3205, -0.0446,  0.1512,  0.3484, -0.3593,
         -0.0053, -0.0880, -0.0034,  0.0471,  0.1062,  0.0647,  0.1003, -0.1662,
          0.1835,  0.2446,  0.1562,  0.0979, -0.1277, -0.1084,  0.2504, -0.0918,
          0.2982,  0.1960,  0.3618,  0.0362, -0.1243, -0.4399,  0.2573,  0.3990,
          0.2662,  0.0485,  0.2297, -0.0400, -0.1349, -0.2962,  0.0479,  0.2586,
         -0.2093, -0.1266, -0.1831,  0.3116, -0.0575,  0.2781,  0.2515,  0.0608,
          0.2055,  0.1133,  0.2411, -0.1764,  0.0767, -0.1386, -0.3741,  0.0262,
         -0.2066

logits tensor([[ 0.6298, -1.8201, -0.5266,  ..., -0.3707,  0.4771,  1.4649],
        [-0.0710, -1.4718, -0.0976,  ..., -0.6046, -0.0276,  0.0330]],
       device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>) torch.Size([2, 768])
smile_embeddings tensor([[-0.1143,  0.2209, -0.1406, -0.3501,  0.0498,  0.1478, -0.2099, -0.1470,
         -0.0771,  0.2537,  0.0482,  0.0305, -0.2971,  0.0978, -0.3734, -0.0664,
         -0.3776, -0.0152, -0.2942,  0.3119, -0.3962, -0.0392,  0.2307, -0.2858,
          0.1960, -0.0603, -0.0552,  0.1560, -0.0172, -0.0850,  0.2081, -0.1214,
          0.1001,  0.1757,  0.0973,  0.0676, -0.0736, -0.0756,  0.0474, -0.1067,
          0.0814,  0.0372,  0.1826,  0.0949,  0.0306, -0.3274, -0.1299,  0.3422,
         -0.0432, -0.1420,  0.0602, -0.1181, -0.1305, -0.3835,  0.1317,  0.1820,
         -0.2351, -0.0829, -0.2623,  0.1862,  0.2624, -0.1170,  0.0678, -0.0972,
          0.3882,  0.2197,  0.2162, -0.2299, -0.2475, -0.3912, -0.1277, -0.2720,
         -0.1033

In [17]:
#test(model, loss_fn, train_dataloader, test_dataloader, scaler_target, device)